### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
#school_data_complete


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
# District Summary
# Step 1 get counts for # of schools, students, budget,...
total_schools = school_data['school_name'].count()
total_students = student_data['student_name'].count()
total_district_budget = school_data['budget'].sum()


In [3]:

# Step 2 get average math, reading, and overall scores
avg_math_score = student_data['math_score'].mean()
avg_reading_score = student_data['reading_score'].mean()
overall_average_score = (avg_math_score + avg_reading_score)/2

# Step 3 get %
pct_math_df = student_data.loc[student_data['math_score'] >= 70] 
pct_students_pass_math = ((pct_math_df['math_score'].count())/total_students)*100
pct_reading_df = student_data.loc[student_data['reading_score'] >= 70] 
pct_students_pass_reading = ((pct_reading_df['reading_score'].count())/total_students)*100



In [4]:
# Step 4 put variables above into summary_table dataframe
summary_table = pd.DataFrame.from_records([{"Total Number of Schools": f"{total_schools: ,.0f}", 
                              "Total Number of Students": f"{total_students:,.0f}",
                              "Total District Budget": f"{total_district_budget:,.0f}",
                              "Average Math Score": f"{avg_math_score:.2f}%",
                              "Average Reading Score": f"{avg_reading_score:.2f}%",
                              "Overall Average Score": f"{overall_average_score:.2f}%",
                              "Math Passing Rate":  f"{pct_students_pass_math:.2f}%",
                              "Reading Passing Rate": f"{pct_students_pass_reading:.2f}%"}])
                              
                              
                             
summary_table

,Total Number of Schools,Total Number of Students,Total District Budget,Average Math Score,Average Reading Score,Overall Average Score,Math Passing Rate,Reading Passing Rate
0,15,"39,170","24,649,428",78.99%,81.88%,80.43%,74.98%,85.81%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [5]:
# Step 1: create dataframe with data from school_data
school_summary = school_data_complete[['school_name', 'type', 'size', 'budget', 'math_score', 'reading_score']]
#school_summary

In [6]:
# Step 2: create columns for per_student_budget, passing_math, passing_reading, and passing_overall
per_student_budget = school_summary["budget"]/school_summary["size"]
passing_math_array = np.where(school_summary["math_score"] >= 70, 1, 0)
passing_reading_array = np.where(school_summary["reading_score"] >= 70, 1, 0)
passing_overall_array = np.where(((school_summary["math_score"] + school_summary["reading_score"])/2) >= 70, 1, 0)

# Step 3: change data type in arrays to integer
passing_math_array_int = passing_math_array.astype(int)
passing_reading_array_int = passing_reading_array.astype(int)
passing_overall_array_int = passing_overall_array.astype(int)

# Step 4: convert arrays to series
passing_math = pd.Series(passing_math_array_int)
passing_reading = pd.Series(passing_reading_array_int)
passing_overall = pd.Series(passing_overall_array_int)

In [9]:
# Step 5: add series to dataframe
school_summary.loc[:,"per_student_budget"] = per_student_budget
school_summary.loc[:,"passing_math"] = passing_math
school_summary.loc[:,"passing_reading"] = passing_reading
school_summary.loc[:,"passing_overall"] = passing_overall

# Step 6: group results by school, and get sums and averages
school_summary_one = school_summary.groupby(["school_name", "type", "size", "budget", "per_student_budget"], as_index = False).agg(
    {
        'math_score':'mean',    
        'reading_score': 'mean',
        'passing_math': 'sum',
        'passing_reading': 'sum',
        'passing_overall': 'sum'
    }
)
#school_summary_one.head()

## Top Performing Schools (By Passing Rate)

In [10]:
# Step 7: with dataset now having one row for each school, calculate % for passing math, reading, overall
school_summary_one["pct_passing_math"] = (school_summary_one["passing_math"]/school_summary_one["size"])*100
school_summary_one["pct_passing_reading"] = (school_summary_one["passing_reading"]/school_summary_one["size"])*100
school_summary_one["pct_passing_overall"] = (school_summary_one["pct_passing_reading"]+school_summary_one["pct_passing_math"])/2

# Step 8: create new dataset to show only percent fields
school_summary_two = school_summary_one.drop(['passing_math', 'passing_reading', 'passing_overall'], axis=1)
#school_summary_two.head()



* Sort and display the top five schools in overall passing rate

In [13]:
school_summary_two = school_summary_two.sort_values(by=["pct_passing_overall"], ascending = False)
school_summary_two.head(5).style.format({
    'budget': '${:,.0f}'.format,
    'per_student_budget': '${:,.2f}'.format,
    'math_score': '{:.2f}'.format,    
    'reading_score': '{:.2f}'.format,
    'pct_passing_math': '{:.2f}%'.format,    
    'pct_passing_reading': '{:.2f}%'.format,    
    'pct_passing_overall': '{:.2f}%'.format,    
})

,school_name,type,size,budget,per_student_budget,math_score,reading_score,pct_passing_math,pct_passing_reading,pct_passing_overall
1,Cabrera High School,Charter,1858,"$1,081,356",$582.00,83.06,83.98,94.13%,97.04%,95.59%
12,Thomas High School,Charter,1635,"$1,043,130",$638.00,83.42,83.85,93.27%,97.31%,95.29%
9,Pena High School,Charter,962,"$585,858",$609.00,83.84,84.04,94.59%,95.95%,95.27%
4,Griffin High School,Charter,1468,"$917,500",$625.00,83.35,83.82,93.39%,97.14%,95.27%
13,Wilson High School,Charter,2283,"$1,319,574",$578.00,83.27,83.99,93.87%,96.54%,95.20%


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [14]:
school_summary_two = school_summary_two.sort_values(by=["pct_passing_overall"])
school_summary_two.head(5).style.format({
    'budget': '${:,.0f}'.format,
    'per_student_budget': '${:,.2f}'.format,
    'math_score': '{:.2f}'.format,    
    'reading_score': '{:.2f}'.format,
    'pct_passing_math': '{:.2f}%'.format,    
    'pct_passing_reading': '{:.2f}%'.format,    
    'pct_passing_overall': '{:.2f}%'.format,    
})

,school_name,type,size,budget,per_student_budget,math_score,reading_score,pct_passing_math,pct_passing_reading,pct_passing_overall
10,Rodriguez High School,District,3999,"$2,547,363",$637.00,76.84,80.74,66.37%,80.22%,73.29%
2,Figueroa High School,District,2949,"$1,884,411",$639.00,76.71,81.16,65.99%,80.74%,73.36%
7,Huang High School,District,2917,"$1,910,635",$655.00,76.63,81.18,65.68%,81.32%,73.50%
8,Johnson High School,District,4761,"$3,094,650",$650.00,77.07,80.97,66.06%,81.22%,73.64%
3,Ford High School,District,2739,"$1,763,916",$644.00,77.10,80.75,68.31%,79.30%,73.80%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [15]:
# Step 1:  Create new columns for school_summary_math
school_summary_math = school_data_complete
school_summary_math['9th'] = np.where(school_data_complete['grade'] == '9th', school_data_complete['math_score'], np.NaN)
school_summary_math['10th'] = np.where(school_data_complete['grade'] == '10th', school_data_complete['math_score'], np.NaN)
school_summary_math['11th'] = np.where(school_data_complete['grade'] == '11th', school_data_complete['math_score'], np.NaN)
school_summary_math['12th'] = np.where(school_data_complete['grade'] == '12th', school_data_complete['math_score'], np.NaN)
#school_summary_math

In [16]:
# Step 2: Group by school_name
school_summary_math = school_summary_math.groupby(["school_name"]).agg(
    {
        '9th':'mean', 
        '10th': 'mean',
        '11th': 'mean',
        '12th': 'mean',
    }
)
school_summary_math

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [17]:
# Step 1:  Create new columns for school_summary_math
school_summary_reading = school_data_complete
school_summary_reading['9th'] = np.where(school_data_complete['grade'] == '9th', school_data_complete['reading_score'], np.NaN)
school_summary_reading['10th'] = np.where(school_data_complete['grade'] == '10th', school_data_complete['reading_score'], np.NaN)
school_summary_reading['11th'] = np.where(school_data_complete['grade'] == '11th', school_data_complete['reading_score'], np.NaN)
school_summary_reading['12th'] = np.where(school_data_complete['grade'] == '12th', school_data_complete['reading_score'], np.NaN)
#school_summary_reading

In [18]:
# Step 2: Group by school_name
school_summary_reading = school_summary_reading.groupby(["school_name"]).agg(
    {
        '9th':'mean', 
        '10th': 'mean',
        '11th': 'mean',
        '12th': 'mean',
    }
)
school_summary_reading

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [19]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [20]:
# Create dataframe for student scores by per pupil spending
school_summary_spending = school_summary_one
school_summary_spending["Spending Ranges (Per Student)"] = pd.cut(school_summary_spending["per_student_budget"], spending_bins, labels=group_names)
#school_summary_spending

In [21]:
# Step 2: Group by spending per student
school_summary_spending = school_summary_spending.groupby(["Spending Ranges (Per Student)"]).agg(
    {
        'math_score':'mean', 
        'reading_score': 'mean',
        'pct_passing_math': 'mean',
        'pct_passing_reading': 'mean',
        'pct_passing_overall': 'mean',

    }
)
school_summary_spending

,math_score,reading_score,pct_passing_math,pct_passing_reading,pct_passing_overall
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


## Scores by School Size

* Perform the same operations as above, based on school size.

In [22]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [23]:
# Step 1:  Create dataframe for student scores by per pupil spending
school_summary_size = school_summary_one
school_summary_size["School Size"] = pd.cut(school_summary_size["size"], size_bins, labels=group_names)
#school_summary_size

In [24]:
# Step 2: Group by spending per student
school_summary_size = school_summary_size.groupby(["School Size"]).agg(
    {
        'math_score':'mean', 
        'reading_score': 'mean',
        'pct_passing_math': 'mean',
        'pct_passing_reading': 'mean',
        'pct_passing_overall': 'mean',

    }
)
school_summary_size

,math_score,reading_score,pct_passing_math,pct_passing_reading,pct_passing_overall
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type

* Perform the same operations as above, based on school type.

In [25]:
# Step 1: Create dataframe for summary by school type
school_summary_type = school_summary_one
# Step 2: Group by spending per student
school_summary_type = school_summary_type.groupby(["type"]).agg(
    {
        'math_score':'mean', 
        'reading_score': 'mean',
        'pct_passing_math': 'mean',
        'pct_passing_reading': 'mean',
        'pct_passing_overall': 'mean',

    }
)
school_summary_type

,math_score,reading_score,pct_passing_math,pct_passing_reading,pct_passing_overall
type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
